In [167]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [168]:
CONFIG = 'mar17'
GAME_NAME = 'large_game_2'
N_SEEDS = 5
overrides = f'--br_portfolio_path mar17 --eval_every 1_000_000 --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000"'

# Base
dispatch.dispatch_experiments(CONFIG, base_job_name='mar23', game_name=f'{GAME_NAME}', overrides = overrides, n_seeds=N_SEEDS)

# Hidden demands
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_hide', overrides = overrides, n_seeds=N_SEEDS)

# Reserve prices
# TODO: Holding off for now b/c rounds issue
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_low_reserve', overrides = overrides, n_seeds=N_SEEDS)
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_same_reserve', overrides = overrides, n_seeds=N_SEEDS)

# Activity
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_no_activity', overrides = overrides, n_seeds=N_SEEDS)
# Undersell
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_no_undersell', overrides = overrides, n_seeds=N_SEEDS)

# Initial pricing experiment, on the base game
dispatch.dispatch_experiments(CONFIG, base_job_name='mar23', game_name=f'{GAME_NAME}_pricing05', overrides = overrides, n_seeds=N_SEEDS)

Submitted batch job 13834
Submitted batch job 13835
Submitted batch job 13836
Submitted batch job 13837
Submitted batch job 13838
Submitted batch job 13839
Submitted batch job 13840
Submitted batch job 13841
Submitted batch job 13842
Submitted batch job 13843
Submitted batch job 13844
Submitted batch job 13845
Submitted batch job 13846
Submitted batch job 13847
Submitted batch job 13848
Submitted batch job 13849
Submitted batch job 13850
Submitted batch job 13851
Submitted batch job 13852
Submitted batch job 13853


In [169]:
CONFIG = 'mar17'
GAME_NAME = 'large_game_3b'
N_SEEDS = 5
overrides = f'--br_portfolio_path mar17 --eval_every 1_000_000 --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000"'

# Base
dispatch.dispatch_experiments(CONFIG, base_job_name='mar22_3', game_name=f'{GAME_NAME}', overrides = overrides, n_seeds=N_SEEDS)

# Hidden demands
dispatch.dispatch_experiments(CONFIG, base_job_name='mar22_3', game_name=f'{GAME_NAME}_hide', overrides = overrides, n_seeds=N_SEEDS)

# Reserve prices
# TODO: Holding off for now b/c rounds issue
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_low_reserve', overrides = overrides, n_seeds=N_SEEDS)
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_same_reserve', overrides = overrides, n_seeds=N_SEEDS)

# Activity
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_no_activity', overrides = overrides, n_seeds=N_SEEDS)
# Undersell
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_no_undersell', overrides = overrides, n_seeds=N_SEEDS)

# Initial pricing experiment, on the base game
dispatch.dispatch_experiments(CONFIG, base_job_name='mar22_3', game_name=f'{GAME_NAME}_pricing05', overrides = overrides, n_seeds=N_SEEDS)

Submitted batch job 13854
Submitted batch job 13855
Submitted batch job 13856
Submitted batch job 13857
Submitted batch job 13858
Submitted batch job 13859
Submitted batch job 13860
Submitted batch job 13861
Submitted batch job 13862
Submitted batch job 13863
Submitted batch job 13864
Submitted batch job 13865
Submitted batch job 13866
Submitted batch job 13867
Submitted batch job 13868
Submitted batch job 13869
Submitted batch job 13870
Submitted batch job 13871
Submitted batch job 13872
Submitted batch job 13873
Submitted batch job 13874
Submitted batch job 13875
Submitted batch job 13876
Submitted batch job 13877
Submitted batch job 13878
Submitted batch job 13879
Submitted batch job 13880
Submitted batch job 13881
Submitted batch job 13882
Submitted batch job 13883


In [115]:
pks = list(map(str, EquilibriumSolverRunCheckpoint.objects.filter(equilibrium_solver_run__experiment__name='infocompare4', equilibrium_solver_run__game__name='large_game_2.json', t=1e7, equilibrium_solver_run__name__contains='mlp').values_list('pk', flat=True)))
CONFIG = 'feb22'
GAME_NAME = 'large_game_2'
N_SEEDS = 1
overrides = f'--br_portfolio_path feb22 --eval_every 1_000_000 --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000" --rnr_player 0 --rnr_checkpoints {" ".join(pks)}'

dispatch.dispatch_experiments(CONFIG, base_job_name='rnrtest', game_name=GAME_NAME, overrides = overrides, n_seeds=N_SEEDS)
# # dispatch.dispatch_experiments('feb22', base_job_name='infocompare3', game_name='large_game_2_hide', overrides = overrides, n_seeds=5)

# dispatch.dispatch_experiments('mar14', base_job_name='mar15_brier_lr', game_name='large_game_2', overrides = overrides, n_seeds=1)
# dispatch.dispatch_experiments('mar10', base_job_name='switch_and_brier3', game_name='large_game_2_switch10', overrides = overrides, n_seeds=1)
# dispatch.dispatch_experiments('feb22', base_job_name='switch_and_brier3_old_config', game_name='large_game_2_switch10', overrides = overrides, n_seeds=1)

# dispatch.dispatch_experiments('feb22', base_job_name='mar14_very_large_game_2', game_name='very_large_game_2', overrides = overrides, n_seeds=1)
# dispatch.dispatch_experiments('feb22', base_job_name='mar14_large_game_3', game_name='large_game_3', overrides = overrides, n_seeds=5)
# dispatch.dispatch_experiments('feb22', base_job_name='mar14_large_game_3', game_name='large_game_3_hide', overrides = overrides, n_seeds=5)
# dispatch.dispatch_experiments('feb22', base_job_name='tiny_game_Test', game_name='tiny', overrides = overrides, n_seeds=1)

Submitted batch job 12737


In [135]:
# !squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "rnrtest" | awk '{print $1}' | xargs scancel

In [176]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R"

             JOBID                                                              NAME ST       TIME  NODES NODELIST(REASON)


In [175]:
!scancel -u ubuntu

In [ ]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

In [ ]:
BestResponse.objects.last()

In [ ]:
Experiment.objects.all()

In [37]:
!sacct --starttime 2022-03-11 --format=User,JobID,Jobname,partition,state,time,start,end,elapsed,MaxRss,MaxVMSize,nnodes,ncpus,nodelist

Slurm accounting storage is disabled


In [50]:
from open_spiel.python.examples.ubc_plotting_utils import parse_run
from auctions.webutils import *

def find_best_checkpoint(run, max_t=None):
    ev_df = parse_run(run, max_t)
    best_t = ev_df.groupby('t')['ApproxNashConv'].first().idxmin()
    nash_conv_by_t = ev_df.groupby('t')['ApproxNashConv'].first()
    best_checkpoint = get_checkpoint(run, t=best_t)
    return nash_conv_by_t, best_checkpoint, nash_conv_by_t.min()


run = EquilibriumSolverRun.objects.last()
find_best_checkpoint(run)

(t
 1          215.790510
 1000000     17.236075
 2000000     14.384932
 3000000     19.374879
 4000000     15.771433
 5000000     13.261669
 6000000      9.768694
 Name: ApproxNashConv, dtype: float64,
 <EquilibriumSolverRunCheckpoint: large_game_2_switch-feb22mlp-100 Iteration 6000000>,
 9.768694201495364)

In [80]:
Experiment.objects.get(name='large_game_2_no_activity.json').delete()

DoesNotExist: Experiment matching query does not exist.

In [125]:
Game.objects.get(name='large_game_2_pricing_extreme.json').delete()

DoesNotExist: Game matching query does not exist.

In [137]:
!python /apps/open_spiel/web/auctions/manage.py nfsp --seed 109 --filename rounds.json --network_config_file mar17/lstm --experiment_name lstmtest --job_name "lstm_for_greg" --dispatch_br false --br_portfolio_path feb22 --eval_every 10_000 --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000" --report_freq 10_000 --overwrite_db true

[autoreload of open_spiel.python.examples.ubc_br failed: Traceback (most recent call last):
  File "/home/ubuntu/.pyenv/versions/3.8.2/envs/venv/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/home/ubuntu/.pyenv/versions/3.8.2/envs/venv/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/home/ubuntu/.pyenv/versions/3.8.2/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 783, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/apps/open_spiel/open_spiel/python/examples/ubc_br.py", line 21, in <module>
    from open_spiel.python.examples.ubc_utils import check_on_q_values, make_dqn_kwargs_from_config, fix_seeds, pretty_time, g

[2022-03-23 01:03:36,952] {ubc_utils.py:112} INFO - Setting numpy and torch seed to 109
[2022-03-23 01:03:36,953] {ubc_utils.py:382} INFO - Reading config from /apps/open_spiel/notebooks/configs/mar17/lstm.yml
[2022-03-23 01:03:36,961] {nfsp.py:132} INFO - Network params: {'add_explore_transitions': False, 'anticipatory_param': 0.1, 'batch_size': 1024, 'epsilon_end': 0.05, 'epsilon_start': 0.9, 'learn_every': 16, 'loss_str': 'mse', 'sl_loss_str': 'cross_entropy', 'min_buffer_size_to_learn': 1000, 'optimizer_str': 'sgd', 'replay_buffer_capacity': 50000, 'reservoir_buffer_capacity': 2000000, 'rl_learning_rate': 0.01, 'rl_model': 'recurrent', 'rl_model_args': {'hidden_size': 128, 'num_layers': 2, 'rnn_model': 'lstm'}, 'sl_learning_rate': 0.01, 'sl_model': 'recurrent', 'sl_model_args': {'hidden_size': 128, 'num_layers': 2, 'rnn_model': 'lstm'}, 'update_target_network_every': 1000, 'double_dqn': True, 'device': 'cuda', 'num_training_episodes': 10000000}
[2022-03-23 01:03:36,961] {nfsp.py:13


KeyboardInterrupt



In [152]:
BestResponse.objects.last().created < datetime.datetime.now(tz=pytz.UTC)


True

In [141]:
datetime.now()

AttributeError: module 'datetime' has no attribute 'now'

In [151]:
import pytz


datetime.datetime(2022, 3, 23, 4, 33, 3, 722237, tzinfo=<UTC>)

In [154]:
datetime.datetime(2022, 3, 23, 4, 33, 3, 722237, tzinfo=pytz.UTC)


datetime.datetime(2022, 3, 23, 4, 33, 3, 722237, tzinfo=<UTC>)

In [156]:
import torch
torch.ones(1000)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 